In [ ]:
imports_path = ".\\imports.ipynb"
tableGAN_path = ".\\tableGAN.ipynb"
utils_path = ".\\utils.ipynb"

%run "$imports_path"

In [ ]:
%run "$utils_path"
%run "$tableGAN_path"

image_dir = ".\\Images"
model_dir = ".\\Model\\tableGAN_ckpt\\compare_pred_on_synthetic_adult\\"
dataset_dir = ".\\Datasets\\"
dataset_train_path = os.path.join(dataset_dir, "df_adult_edited_train.csv")
dataset_test_path = os.path.join(dataset_dir, "df_adult_edited_test.csv")
dataset_gen_dir = os.path.join(dataset_dir, "Generated_for_pred_eval_adult//")

data= pd.read_csv('Datasets\\df_adult_edited.csv')
discrete_columns = data.columns[data.dtypes == "object"]
data_train, data_test = train_test_split(data, test_size=0.4, random_state=1)
print(f"Train size: {data_train.shape[0]}, Test size: {data_test.shape[0]}")

data_train.to_csv(dataset_train_path)
data_test.to_csv(dataset_test_path)

In [ ]:
n_epochs = 100
n_critic = 10
adam_lr = 0.002
adam_beta1 = 0.5
noise_discrete_unif_max = 0

batch_size = 500
ckpt_every = 1
loss_plot_update_every = 10

n_synthetic_datasets = 10
n_synthetic_datasets_existing = 0

retrain = False

In [4]:
%run "$tableGAN_path"
tg = TableGAN(data, n_critic = n_critic, adam_lr = adam_lr, adam_beta1 = adam_beta1,
              quantile_transformation_int = False, quantile_rand_transformation = False,
             noise_discrete_unif_max = noise_discrete_unif_max)

tg_qt = TableGAN(data, n_critic = n_critic, adam_lr = adam_lr, adam_beta1 = adam_beta1,
              quantile_transformation_int = True, quantile_rand_transformation = False,
             noise_discrete_unif_max = noise_discrete_unif_max)

tg_qtr = TableGAN(data, n_critic = n_critic, adam_lr = adam_lr, adam_beta1 = adam_beta1,
              quantile_transformation_int = True, quantile_rand_transformation = True,
             noise_discrete_unif_max = noise_discrete_unif_max)

In [5]:
if retrain:
    generate_multiple_datasets(tg, dataset_gen_dir, n_synthetic_datasets, n_epochs, subfolder = "tabGAN",
                              n_synthetic_datasets_existing=n_synthetic_datasets_existing)

In [6]:
if retrain:
    generate_multiple_datasets(tg_qt, dataset_gen_dir, n_synthetic_datasets, n_epochs, subfolder = "tabGAN-qt",
                              n_synthetic_datasets_existing=n_synthetic_datasets_existing)

In [7]:
if retrain:
    generate_multiple_datasets(tg_qtr, dataset_gen_dir, n_synthetic_datasets, n_epochs, subfolder = "tabGAN-qtr",
                              n_synthetic_datasets_existing=n_synthetic_datasets_existing)

In [8]:
if retrain:
    dataset_dir_ctgan = os.path.join(dataset_gen_dir, "CTGAN") 
    os.makedirs(dataset_dir_ctgan, exist_ok = True)
    for i in tqdm(range(n_synthetic_datasets), desc = "Generated datasets"):

        ctgan = CTGANSynthesizer(epochs=n_epochs, discriminator_steps=n_critic)
        ctgan.fit(data_train, tg.columns_discrete)
        fake_train = ctgan.sample(data_train.shape[0])
        fake_train.to_csv(os.path.join(dataset_dir_ctgan, f"gen{i}.csv"))
        del ctgan

In [9]:
%run "$utils_path"

subfolders = ["tabGAN", "tabGAN-qt", "tabGAN-qtr", "CTGAN", "TGAN", "tabFairGAN", "tabFairGAN-mod"]
evaluate_tabGAN_through_prediction(data_train, data_test, dataset_gen_dir, subfolders, n_synthetic_datasets = 5)

  0%|          | 0/36 [00:00<?, ?it/s]

,Model,Accuracy,AUC,SD Accuracy,SD AUC
0,Train dataset,0.82856,0.870968,0,0
1,tabGAN,0.808614,0.842037,0.008287,0.005719
2,tabGAN-qt,0.813221,0.839334,0.004017,0.00671
3,tabGAN-qtr,0.804775,0.828234,0.007455,0.015613
4,CTGAN,0.806787,0.821325,0.005266,0.004287
5,TGAN,0.71005,0.529143,0.024137,0.103481
6,tabFairGAN,0.759432,0.572379,0.004786,0.035611
7,tabFairGAN-mod,0.798403,0.815467,0.001824,0.005832


In [71]:
evaluate_tabGAN_through_prediction(data_train, data_test, dataset_gen_dir, subfolders = ["tabGAN-qtr"], n_synthetic_datasets = 10)

  0%|          | 0/11 [00:00<?, ?it/s]

,Model,Accuracy,AUC,SD Accuracy,SD AUC
0,Train dataset,0.82856,0.870968,0,0
1,tabGAN-qtr,0.806841,0.834713,0.007901,0.013756


In [6]:
tg_qt = TableGAN(data, n_critic = n_critic, adam_lr = adam_lr, adam_beta1 = adam_beta1,
              quantile_transformation_int = True, quantile_rand_transformation = False,
             noise_discrete_unif_max = noise_discrete_unif_max,
                gumbel_temperature = 0.5)

dataset_epochs_comparison_dir = os.path.join(dataset_gen_dir, "epoch_comparison")
n_epochs_vec = np.arange(1, 26).tolist() + np.arange(30,101, 10).tolist()
path_finished_epochs_counter = ".//Python_objects//existing_n_epochs_tracker.pkl"
generate_multiple_datasets_for_multiple_epochs(tg_qt, dataset_dir = dataset_epochs_comparison_dir,
                                               n_synthetic_datasets = 5, n_epochs_vec = n_epochs_vec,
                                              path_finished_epochs_counter = path_finished_epochs_counter,
                                              redo_n_epochs_vec=[])

Epoch subfolder creation: 0it [00:00, ?it/s]

In [ ]:
result = evaluate_n_epochs_through_prediction(data_train, data_test, dataset_dir = dataset_epochs_comparison_dir,
                                     n_epochs_vec = n_epochs_vec, n_synthetic_datasets = 5,
                                    save_dir = image_dir, save_path = "predictive_performance_against_n_epochs.jpg")

  0%|          | 0/165 [00:00<?, ?it/s]